In [1]:
import requests
import osmnx as ox
import pandas as pd
import geopandas as gpd
import osm2geojson

from shapely.geometry import shape
from shapely.geometry import MultiPolygon
from shapely.geometry import shape, box
from shapely.geometry import Polygon

In [2]:
STREETS_FILE_URL = "https://drive.google.com/file/d/1bUT1E-QSbG1vpSNM2dOG2-LEVXSrPdo3/view?usp=sharing"
BUFFER_SIZE = 10
projection = 32638
new_crs = "EPSG:4326"


In [3]:
# territory_geojson = 'vaska.geojson'
# territory_gdf = gpd.read_file(territory_geojson)
# # territory_gdf = territory_gdf.to_crs(projection)

In [4]:
gdf = gpd.read_file('data/blocks.geojson')
gdf = gdf.to_crs(new_crs)
test = gdf.iloc[24]
territory_gdf = gpd.GeoDataFrame([test])
territory_gdf

,id,geometry
24,24,"POLYGON ((30.21380 59.95809, 30.21405 59.95827..."


In [6]:
def minimum_convex_quadrilateral(geometry):
    if geometry.geom_type == 'Polygon':
        min_rect = geometry.minimum_rotated_rectangle
        return min_rect
    else:
        return None


In [7]:
polygon = gpd.GeoDataFrame()
polygon['geometry'] = territory_gdf['geometry'].apply(minimum_convex_quadrilateral)
# Установить 'geometry' как активную геометрию
polygon = polygon.set_geometry('geometry')

coordinate = ','.join(map(str, [polygon.total_bounds[1], 
                                    polygon.total_bounds[0], 
                                    polygon.total_bounds[3], 
                                    polygon.total_bounds[2]]))
print(coordinate)


59.953601196459516,30.211669778798733,59.96140885432281,30.23069512310009


/var/folders/h8/0wmx2zx90bn60jw8zc1r4qjc0000gn/T/ipykernel_5935/2478268324.py:2: FutureWarning: You are adding a column named 'geometry' to a GeoDataFrame constructed without an active geometry column. Currently, this automatically sets the active geometry column to 'geometry' but in the future that will no longer happen. Instead, either provide geometry to the GeoDataFrame constructor (GeoDataFrame(... geometry=GeoSeries()) or use `set_geometry('geometry')` to explicitly set the active geometry column.
  polygon['geometry'] = territory_gdf['geometry'].apply(minimum_convex_quadrilateral)


## Основной код


In [10]:
def dwn_leisure(coordinate, projection):
    '''
    :param city_name: name of city --> str
    :param projection: epsg code of city crs --> int
    :return: GeoDataFrame
    '''
    overpass_url = "https://maps.mail.ru/osm/tools/overpass/api/interpreter"
    overpass_query = f"""
    [out:json];
        (
            relation["leisure"]({coordinate});
            way["leisure"]({coordinate});
        );
        out geom;
    """
    result = None
    attempts = 0
    while result is None and attempts < 5:
        try:
            result = requests.get(overpass_url, params={'data': overpass_query}).json()
            resp = osm2geojson.json2geojson(result)
            leisure = gpd.GeoDataFrame.from_features(resp['features']).set_crs(4326).to_crs(projection)
            leisure = leisure.geometry
            print('Leisure objects downloaded')
            return leisure
        except:
            attempts += 1

    print('Failed to download leisure objects after 5 attempts')
    return None



In [11]:
leisure=dwn_leisure(coordinate, projection)

Leisure objects downloaded


In [12]:
leisure

0     POLYGON ((-321163.075 6738694.671, -321174.343...
1     POLYGON ((-320388.973 6739228.779, -320393.966...
2     POLYGON ((-320446.689 6739108.662, -320423.238...
3     POLYGON ((-320978.659 6739006.459, -320953.095...
4     POLYGON ((-321083.345 6739056.545, -321103.399...
                            ...                        
83    POLYGON ((-320837.111 6738973.655, -320832.463...
84    POLYGON ((-320383.857 6738637.707, -320390.600...
85    POLYGON ((-320393.797 6738612.253, -320391.461...
86    MULTIPOLYGON (((-320447.576 6739170.197, -3204...
87    MULTIPOLYGON (((-320960.955 6738819.451, -3209...
Name: geometry, Length: 88, dtype: geometry

In [13]:
def dwn_other(coordinate, projection):
    '''
    :param city_name: name of city --> str
    :param projection: epsg code of city crs --> int
    :return: GeoDataFrame
    '''
    overpass_url = "https://maps.mail.ru/osm/tools/overpass/api/interpreter"
    overpass_query = f"""
    [out:json];
            
        (
            relation["man_made"]({coordinate});
            way["man_made"]({coordinate});
            
            relation["aeroway"]({coordinate});
            way["aeroway"]({coordinate});

            way["military"]({coordinate});
            relation["military"]({coordinate});
        
        );
    out geom;
    """
    
    result = None
    attempts = 0
    while result is None and attempts < 5:
        try:
            result = requests.get(overpass_url, params={'data': overpass_query}).json()
            resp = osm2geojson.json2geojson(result)
            other = gpd.GeoDataFrame.from_features(resp['features']).set_crs(4326).to_crs(projection)
            other = other.geometry
            print('Other objects downloaded')
            return other
        except:
            attempts += 1

    print('Failed to download other objects after 5 attempts')
    return None



In [14]:
other=dwn_other(coordinate, projection)

Other objects downloaded


In [15]:
other

0    POLYGON ((-320742.352 6739020.545, -320725.733...
1    POLYGON ((-320558.240 6739051.883, -320551.740...
2    LINESTRING (-320766.884 6739464.509, -320802.0...
3    LINESTRING (-321004.458 6739517.851, -321020.3...
4    LINESTRING (-321236.294 6739315.329, -321208.7...
5    POLYGON ((-321061.927 6739371.823, -321056.151...
6    POLYGON ((-320768.543 6739484.124, -320769.365...
7    POLYGON ((-320840.894 6739501.111, -320836.695...
8    POLYGON ((-321054.787 6739401.959, -321070.852...
9    POLYGON ((-320573.697 6738832.649, -320570.665...
Name: geometry, dtype: geometry

In [66]:
def dwn_landuse(coordinate, projection):
    '''
    :param city_name: name of city --> str
    :param projection: epsg code of city crs --> int
    :return: GeoDataFrame
    '''
    overpass_url = "https://maps.mail.ru/osm/tools/overpass/api/interpreter"
    overpass_query = f"""
    [out:json];
        (
        relation["landuse"]["landuse"!~"residential"]({coordinate});
        way["landuse"]["landuse"!~"residential"]({coordinate});
        );
    (._;);
    out geom;
    """

    result = None
    attempts = 0
    while result is None and attempts < 5:
        try:
            result = requests.get(overpass_url, params={'data': overpass_query}).json()
            resp = osm2geojson.json2geojson(result)
            landuse = gpd.GeoDataFrame.from_features(resp['features']).set_crs(4326).to_crs(projection)
            landuse = landuse.geometry
            print('Landuse objects downloaded')
            return landuse
        except:
            attempts += 1

    print('Failed to download landuse objects after 5 attempts')
    return None

 

In [67]:
landuse=dwn_landuse(coordinate, projection)
landuse

Landuse objects downloaded


0      POLYGON ((-320187.663 6739288.972, -320180.118...
1      POLYGON ((-320165.663 6739283.870, -320166.871...
2      POLYGON ((-320244.397 6739234.234, -320245.044...
3      POLYGON ((-320537.707 6738792.839, -320630.824...
4      POLYGON ((-321341.524 6738932.317, -321341.391...
                             ...                        
113    POLYGON ((-320737.845 6738937.993, -320725.728...
114    POLYGON ((-320723.349 6739012.324, -320717.602...
115    POLYGON ((-320766.808 6738982.812, -320753.972...
116    POLYGON ((-320958.027 6738838.870, -320925.169...
117    MULTIPOLYGON (((-320328.280 6739209.465, -3203...
Name: geometry, Length: 118, dtype: geometry

In [18]:
landuse.explore()

In [19]:
def dwn_amenity(coordinate, projection):
    '''
    :param city_name: name of city --> str
    :param projection: epsg code of city crs --> int
    :return: GeoDataFrame
    '''
    overpass_url = "https://maps.mail.ru/osm/tools/overpass/api/interpreter"
    overpass_query = f"""
    [out:json];
    (
        relation["amenity"]({coordinate});
        way["amenity"]({coordinate});
    );
    out geom;
    """
    result = None
    attempts = 0
    while result is None and attempts < 5:
        try:
            result = requests.get(overpass_url, params={'data': overpass_query}).json()
            resp = osm2geojson.json2geojson(result)
            amenity = gpd.GeoDataFrame.from_features(resp['features']).set_crs(4326).to_crs(projection)
            amenity = amenity.geometry
            print('Amenity objects downloaded')
            return amenity
        except:
            attempts += 1

    print('Failed to download amenity objects after 5 attempts')
    return None


In [20]:
amenity=dwn_amenity(coordinate, projection)
amenity


Amenity objects downloaded


0     POLYGON ((-320636.359 6739130.304, -320631.744...
1     POLYGON ((-320459.232 6739231.517, -320442.245...
2     POLYGON ((-320407.065 6738649.177, -320401.876...
3     POLYGON ((-320633.830 6739206.182, -320610.670...
4     POLYGON ((-320353.255 6739298.358, -320316.758...
5     POLYGON ((-320472.943 6739347.751, -320469.029...
6     POLYGON ((-320637.239 6739328.359, -320629.039...
7     POLYGON ((-321366.312 6739033.297, -321308.289...
8     POLYGON ((-320723.395 6738779.179, -320672.517...
9     POLYGON ((-320913.491 6738703.047, -320947.373...
10    POLYGON ((-320576.535 6739131.986, -320563.092...
11    POLYGON ((-320609.822 6739289.560, -320608.021...
12    POLYGON ((-321157.373 6739065.304, -321147.385...
13    POLYGON ((-320741.862 6739340.436, -320740.276...
14    POLYGON ((-320950.693 6739315.172, -320957.986...
15    POLYGON ((-320487.360 6739360.165, -320502.373...
16    POLYGON ((-320571.856 6739342.107, -320592.425...
17    POLYGON ((-320399.724 6739356.442, -320416

In [21]:
amenity.explore()

In [38]:
def dwn_buildings(coordinate, projection):
    '''
    :param city_name: name of city --> str
    :param projection: epsg code of city crs --> int
    :return: GeoDataFrame
    '''
    overpass_url = "https://maps.mail.ru/osm/tools/overpass/api/interpreter"
    overpass_query = f"""
    [out:json];
        (
            relation["building"]({coordinate});
            way["building"]({coordinate});
        );
    out geom;
    """
    result = None
    attempts = 0
    while result is None and attempts < 5:
        try:
            result = requests.get(overpass_url, params={'data': overpass_query}).json()
            resp = osm2geojson.json2geojson(result)
            buildings = gpd.GeoDataFrame.from_features(resp['features']).set_crs(4326).to_crs(projection)
            buildings = buildings.geometry
            print('Buildings objects downloaded')
            return buildings
        except:
            attempts += 1

    print('Failed to download buildings objects after 5 attempts')
    return None

In [39]:
buildings=dwn_buildings(coordinate, projection)
buildings

Buildings objects downloaded


0      POLYGON ((-320624.659 6738518.138, -320635.821...
1      POLYGON ((-320363.733 6738586.249, -320364.980...
2      POLYGON ((-320493.456 6738791.698, -320491.969...
3      POLYGON ((-320371.640 6738704.242, -320382.562...
4      POLYGON ((-320569.141 6738656.137, -320570.209...
                             ...                        
181    MULTIPOLYGON (((-321041.404 6738712.995, -3210...
182    MULTIPOLYGON (((-320895.792 6739162.558, -3209...
183    MULTIPOLYGON (((-320218.495 6739044.262, -3202...
184    MULTIPOLYGON (((-321134.301 6738700.077, -3211...
185    MULTIPOLYGON (((-320430.970 6738718.936, -3204...
Name: geometry, Length: 186, dtype: geometry

In [24]:
buildings.explore()

In [108]:
def dwn_natural(coordinate, projection):
    '''
    :param city_name: name of city --> str
    :param projection: epsg code of city crs --> int
    :return: GeoDataFrame
    '''
    overpass_url = "https://maps.mail.ru/osm/tools/overpass/api/interpreter"
    overpass_query = f"""
    [out:json];
    (
        way["natural"]({coordinate});
        relation["natural"]({coordinate});
    );      
    out geom;
    """
    result = None
    attempts = 0
    while result is None and attempts < 5:
        try:
            result = requests.get(overpass_url, params={'data': overpass_query}).json()
            resp = osm2geojson.json2geojson(result)
            natural = gpd.GeoDataFrame.from_features(resp['features']).set_crs(4326).to_crs(projection)
            natural = natural.geometry
            print('Natural objects downloaded')
            return natural
        except:
            attempts += 1

    print('Failed to download natural objects after 5 attempts')
    return None

In [109]:
natural=dwn_natural(coordinate, projection)
natural

Natural objects downloaded


0     LINESTRING (-320168.573 6739365.375, -320208.6...
1     POLYGON ((-321580.132 6738708.496, -321543.138...
2     POLYGON ((-320307.046 6738781.673, -320264.585...
3     POLYGON ((-320315.870 6738735.395, -320320.417...
4     POLYGON ((-320329.690 6738792.830, -320316.612...
5     POLYGON ((-320498.242 6738633.709, -320500.682...
6     POLYGON ((-320373.797 6738657.766, -320372.691...
7     POLYGON ((-320395.279 6738645.059, -320392.238...
8     POLYGON ((-320367.553 6739268.440, -320379.018...
9     POLYGON ((-321246.701 6739294.359, -321246.899...
10    POLYGON ((-321167.508 6739220.479, -321175.475...
11    MULTIPOLYGON (((-321360.245 6738982.623, -3213...
Name: geometry, dtype: geometry

In [99]:
natural.explore()

In [68]:

def dwn_waterway(coordinate, projection):
    '''
    :param city_name: name of city --> str
    :param projection: epsg code of city crs --> int
    :return: GeoDataFrame
    '''
    overpass_url = "https://maps.mail.ru/osm/tools/overpass/api/interpreter"
    overpass_query = f"""
    [out:json];
    (
        way["waterway"]({coordinate});
        relation["waterway"]({coordinate});
    ); 
    out geom;
    """
    result = None
    attempts = 0
    
    while result is None and attempts < 5:
        try:
            result = requests.get(overpass_url, params={'data': overpass_query}).json()
            resp = osm2geojson.json2geojson(result)
            waterway = gpd.GeoDataFrame.from_features(resp['features']).set_crs(4326).to_crs(projection)
            waterway = waterway.geometry
            print('Waterway objects downloaded')
            return waterway
        except:
            attempts += 1

    print('Failed to download waterway objects after 5 attempts')
    return None  # Возвращаем пустой GeoDataFrame, если не удалось получить данные




In [69]:
waterway=dwn_waterway(coordinate, projection)
waterway

Failed to download waterway objects after 5 attempts


In [87]:
def dwn_roads(coordinate, projection):
    '''
    :param city_name: name of targeted city --> str
    :param projection: epsg code of city crs --> int
    :return: GeoDataFrame
    '''
    overpass_url = "https://maps.mail.ru/osm/tools/overpass/api/interpreter"
    overpass_query = f"""
    [out:json];
    (      
            way["highway"]["highway"!~"path|footway|pedestrian"]({coordinate});
           
            way["railway"]["railway"!~"subway"]({coordinate});
        
    );
    (._;);
    out geom;
    """
    result = None
    attempts = 0
    
    while result is None and attempts < 5:
        try:
            result = requests.get(overpass_url, params={'data': overpass_query}).json()
            resp = osm2geojson.json2geojson(result)
            roads = gpd.GeoDataFrame.from_features(resp['features']).set_crs(4326).to_crs(projection)
            roads = roads.geometry
            print('Roads objects downloaded')
            return roads
        except:
            attempts += 1

    print('Failed to download roads objects after 5 attempts')
    return None  # Возвращаем пустой GeoDataFrame, если не удалось получить данные

In [88]:
roads=dwn_roads(coordinate, projection)

Roads objects downloaded


In [89]:
roads

0      LINESTRING (-320864.069 6738891.965, -320874.2...
1      LINESTRING (-321255.073 6739049.848, -321247.0...
2      LINESTRING (-320809.102 6738917.178, -320812.2...
3      LINESTRING (-320627.429 6738883.317, -320626.0...
4      LINESTRING (-321415.336 6738760.295, -321404.8...
                             ...                        
389    LINESTRING (-320167.979 6739298.243, -320156.9...
390    LINESTRING (-321027.182 6738790.303, -321033.7...
391    LINESTRING (-320974.774 6739196.266, -320994.5...
392    LINESTRING (-320995.364 6739160.345, -320976.8...
393    LINESTRING (-320976.824 6739150.731, -320974.7...
Name: geometry, Length: 394, dtype: geometry

In [90]:
roads.explore()

In [103]:
leisure=dwn_leisure(coordinate, projection)
other=dwn_other(coordinate, projection)
landuse=dwn_landuse(coordinate, projection)
amenity=dwn_amenity(coordinate, projection)
natural=dwn_natural(coordinate, projection)
waterway=dwn_waterway(coordinate, projection)
buildings=dwn_buildings(coordinate, projection)
roads=dwn_roads(coordinate, projection)

Leisure objects downloaded
Other objects downloaded
Landuse objects downloaded
Amenity objects downloaded
Natural objects downloaded
Buildings objects downloaded
Roads objects downloaded


In [64]:
def free_area(city_name, projection, city_blocks_file):
    '''
    :param city_name: name of city --> str
    :param projection: epsg code of city crs --> int
    :param city_blocks_file: file with blocks of city. Attributes: id, geometry --> 'name.geojson' in root
    :return: GeoDataFrame with free space in the city
    '''
    blocks = gpd.read_file(city_blocks_file)
    blocks = gpd.GeoDataFrame.from_features(blocks).set_crs(4326).to_crs(projection)
    print('City blocks downloaded')

    leisure=dwn_leisure(coordinate, projection)
    other=dwn_other(coordinate, projection)
    landuse=dwn_landuse(coordinate, projection)
    amenity=dwn_amenity(coordinate, projection)
    natural=dwn_natural(coordinate, projection)
    waterway=dwn_waterway(coordinate, projection)
    buildings=dwn_buildings(coordinate, projection)
    roads=dwn_roads(coordinate, projection)

    
    occupied_area = [leisure, other, landuse, amenity, natural, waterway, buildings, roads]
    occupied_area = pd.concat(occupied_area)
    occupied_area = gpd.GeoDataFrame(geometry=gpd.GeoSeries(occupied_area))



    print('Clipping objects from blocks')

    polygon = occupied_area.geometry.geom_type == "Polygon"
    multipolygon = occupied_area.geometry.geom_type == "MultiPolygon"
    blocks_new = gpd.overlay(blocks, occupied_area[polygon], how="difference")
    blocks_new = gpd.overlay(blocks_new, occupied_area[multipolygon], how="difference")
    blocks_new = blocks_new.reset_index(drop=False).rename(columns={"index": "block_id"})

    print('Exploding multipolygons into polygons')

    blocks_exploded = gpd.GeoDataFrame(columns=blocks_new.columns)
    for index, row in blocks_new.iterrows():
        if row.geometry.geom_type == 'Polygon':
            blocks_exploded = blocks_exploded.append(row,ignore_index=True)
        if row.geometry.geom_type == 'MultiPolygon':
            multdf = gpd.GeoDataFrame(columns=blocks_new.columns)
            recs = len(row.geometry.geoms)
            multdf = multdf.append([row]*recs,ignore_index=True)
            for geom in range(recs):
                multdf.loc[geom, 'geometry'] = row.geometry[geom]
            blocks_exploded = blocks_exploded.append(multdf,ignore_index=True)
            
    blocks_exploded['max_empty_area']=blocks_exploded.area

    print('Finding max empty area')
    
    blocks_max = blocks_exploded.sort_values(['block_id', 'max_empty_area'], ascending=[True,False])
    blocks_max = blocks_max.drop_duplicates('block_id',keep='first').reset_index(drop=True)
    blocks_max = blocks_max.set_crs(projection).to_crs(4326)

    print('Ready')
    return blocks_max

In [14]:
#here city name in OSM (can be in russian, can be in english), EPSG code of CRS, way to file with city blocks
free_area = free_area('Санкт-Петербург', 32638, '/home/max/jup/Data/Kvartal.geojson') 
free_area.to_file("SPB_free_area.geojson", driver='GeoJSON')

DriverError: /home/max/jup/Data/Kvartal.geojson: No such file or directory

: 

: 

: 

: 